<a href="https://colab.research.google.com/github/tnsharath/AI/blob/master/Signature_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [2]:
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Get data from Drive

In [3]:
shutil.copy("/content/drive/My Drive/colab_work/signature_data.zip", "/content/")

'/content/signature_data.zip'

In [ ]:
!unzip /content/signature_data.zip

## Data preprocessing


In [5]:
def setup_data():
  train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,)
  training_set = train_datagen.flow_from_directory('data/train_set',
                                                 target_size = (128, 128),
                                                 batch_size = 258)
  X_train, y_train = training_set.next()
  #X_train = X_train.reshape(258, 128 * 128 * 3)
  
  return X_train, y_train

In [6]:
# Random Dim
np.random.seed(10)
random_dim = 100

# Models

In [7]:
def get_generator():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(128 * 4 * 4 , input_dim=random_dim))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Reshape((4, 4, 128)))

  model.add(tf.keras.layers.Conv2D(256, 5, padding= 'same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))
  model.add(tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same'))
  model.add(tf.keras.layers.LeakyReLU(0.2))

  model.add(tf.keras.layers.Conv2D(3, 7, activation='tanh', padding= 'same'))
  # model.add(tf.keras.layers.Flatten())

  model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
  return model

In [8]:
get_generator().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2048)              206848    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 2048)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 256)         819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 128)         524416    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 128)         0

In [9]:
def get_discriminator():
    cnn = tf.keras.Sequential()

    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size= 1, input_shape=[128, 128, 3]) )
    cnn.add(tf.keras.layers.LeakyReLU(0.2))
    cnn.add(tf.keras.layers.Dropout(0.3))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3))
    cnn.add(tf.keras.layers.LeakyReLU(0.2))
    cnn.add(tf.keras.layers.Dropout(0.3))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

    cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3))
    cnn.add(tf.keras.layers.LeakyReLU(0.2))
    cnn.add(tf.keras.layers.Dropout(0.3))
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

    cnn.add(tf.keras.layers.Flatten())

    cnn.add(tf.keras.layers.Dense(units=256))
    cnn.add(tf.keras.layers.LeakyReLU(0.2))
    cnn.add(tf.keras.layers.Dropout(0.3))


    cnn.add(tf.keras.layers.Dense(units=128))
    cnn.add(tf.keras.layers.LeakyReLU(0.2))
    cnn.add(tf.keras.layers.Dropout(0.3))

    cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    cnn.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    return cnn
    

In [10]:
get_discriminator().summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 62, 62, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 32)       

In [13]:
def get_gan_network(discriminator, random_dim, generator):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = tf.keras.Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = tf.keras.Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    return gan

In [14]:
# Create a wall of generated images
def plot_generated_images(epoch, generator, examples=25, dim=(5, 5), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 128, 128, 3)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/content/drive/My Drive/colab_work/results/gan_generated_image_epoch_%d.png' % epoch)

In [15]:
def train(epochs=1, batch_size=128):
    # Get the training 
    x_train, y_train= setup_data()

    # Split the training data into batches of size 128
    batch_count = int(x_train.shape[0] / batch_size)

    # Build our GAN netowrk
    generator = get_generator()
    discriminator = get_discriminator()
    gan = get_gan_network(discriminator, random_dim, generator)

    for e in range(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in range(batch_count):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])

            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            generated_images = generator.predict(noise)

            
            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            #discriminator.train_on_batch(image_batch.reshape(128, 128, 128, 3), y_dis[:batch_size])
            discriminator.train_on_batch(image_batch, y_dis[:batch_size])
            discriminator.train_on_batch(generated_images, y_dis[batch_size:])

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
           plot_generated_images(e, generator)

        if e % 200 == 0:
          generator.save_weights("/content/drive/My Drive/colab_work/results/gen_weights.h5")
          discriminator.save_weights("/content/drive/My Drive/colab_work/results/discriminator_weights.h5")


In [ ]:
if __name__ == '__main__':
    train(2000, 128)